In [1]:
# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras

# Import all the necessary for our model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard
from keras.regularizers import l2
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D, LeakyReLU

# Import helper libraries
import numpy as np
import scipy as scipy
import os

from helpers import * 

print(tf.__version__)

Using TensorFlow backend.


1.12.0


## Loading the images 

In [2]:
# Load a set of images
root_dir = "provided/training/"

# Select the directory for the images and load them
image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = len(files) 

print("Loading " + str(n) + " images")
imgs = np.asarray([load_image(image_dir + files[i]) for i in range(n)])

# Select the directory for groundtruth images and load them
gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " groundtruth images")
gt_imgs = np.asarray([load_image(gt_dir + files[i]) for i in range(n)])

Loading 100 images
Loading 100 groundtruth images


In [3]:
imgs.shape

(100, 400, 400, 3)

In [4]:
gt_imgs.shape

(100, 400, 400)

In [5]:
image_size = 400

# We separate the images from the groundtruth images
img_patches = [img_crop(imgs[i], image_size, image_size) for i in range(n)]
gt_patches = [img_crop(gt_imgs[i], image_size, image_size) for i in range(n)]

# Linearize the list and labeling them X and Y
X = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
Y = np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

In [6]:
X.shape

(100, 400, 400, 3)

In [11]:
Y.shape

(100, 400, 400)

## Generating mini-batch and running data augmentation

In [ ]:
def create_minibatch():
    
    # Fix the seed
    np.random.seed(1)
    
    # We define the window size of 72, batch size of 100 (empirically chosen)
    # and patch size should correspond to 16
    w_size = 72
    batch_size = 100
    patch_size = 16
    num_images = 100
    
    while True:
        # Generate one minibatch
        batch_image = np.empty((batch_size, w_size, w_size, 3))
        batch_label = np.empty((batch_size, 2))
        
        for i in range(batch_size):
            
            # Select a random index represnting an image
            random_index = np.random.choice(num_images)
            
            # Width of original image
            width = 400
            
            # Sample a random window from the image
            random_sample = np.random.randint(w_size//2, width - w_size//2, 2)
            
            # Create a sub image of size 72x72
            sampled_image = X[random_index][random_sample[0] - w_size // 2 : random_sample[0] + w_size//2,
                                            random_sample[1] - w_size//2 : random_sample[1] + w_size//2]
                
            # Take its corresponding ground-truth image
            correspond_ground_truth = Y[random_index][random_sample[0] - patch_size//2:random_sample[0] + patch_size//2,
                                                      random_sample[1]-patch_size//2:random_sample[1] + patch_size//2]
            
            # We set in the label depending on the threshold of 0.25
            # The label becomes either 0 or 1 by applying to_categorical with parameter 2
            label = to_categorical((np.array([np.mean(correspond_ground_truth)]) > 0.25) * 1, 2)
            
            # The image augmentation is based on both flipping and rotating (randomly in steps of 45°)
            # Random vertical and horizontal flip
            if np.random.choice(2) == 1:
                sampled_image = np.flipud(sampled_image)
            
            if np.random.choice(2) == 1:
                sampled_image = np.fliplr(sampled_image)
                    
            # Random rotation in steps of 45°
            rotations = [0, 45, 90, 135, 180, 225, 270, 315, 350]
        
            # We select a rotation degree randomly
            rotation_choice = np.random.choice(len(rotations))
            
            # Rotate it using the random value (uses the scipy library)
            sampled_image = scipy.ndimage.rotate(sampled_image, rotations[rotation_choice], order=1,
                                                         reshape=False, mode='reflect')
                        
            # We put in the sub image and its corresponding label before yielding it
            batch_image[i] = sampled_image
            batch_label[i] = label

        # Yield the mini_batch to the model
        yield(batch_image, batch_label)

## Creating the class (Same as in cnn_model.py, but provided here for better readability)

In [ ]:
class cnn_model:
    
    # Initialize the class
    def __init__(self, shape):
        self.shape = shape
        self.model = self.initialize_cnn_model(shape)
    
    def initialize_cnn_model(self, shape):
        
        # INPUT
        # shape     - Size of the input images
        # OUTPUT
        # model    - Compiled CNN
        
        # Define hyperparamters
        KERNEL3 = (3, 3)
        KERNEL5 = (5, 5)
        
        # Define a model
        model = Sequential()
        
        # Add the layers
        # Selection of the model is described in the report
        # We use padding = 'same' to avoid issues with the matrix sizes
        model.add(Conv2D(64, KERNEL5, input_shape = shape, padding='same'))
        model.add(LeakyReLU(alpha=0.1))
        model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(128, KERNEL3, padding='same'))
        model.add(LeakyReLU(alpha=0.1))
        model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(256, KERNEL3, padding='same'))
        model.add(LeakyReLU(alpha=0.1))
        model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(256, KERNEL3, padding='same'))
        model.add(LeakyReLU(alpha=0.1))
        model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(256, KERNEL3, padding='same'))
        model.add(LeakyReLU(alpha=0.1))
        model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(256, KERNEL3, padding='same'))
        model.add(LeakyReLU(alpha=0.1))
        model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
        model.add(Dropout(0.25))
        
        # Flatten it and use regularizers to avoid overfitting
        # The parameters have been chosen empirically
        model.add(Flatten())
        model.add(Dense(128, kernel_regularizer=l2(0.000001), activity_regularizer=l2(0.000001)))
        model.add(LeakyReLU(alpha=0.1))
        model.add(Dropout(0.5))
        
        # Add output layer
        model.add(Dense(2, kernel_regularizer=l2(0.000001), activity_regularizer=l2(0.000001)))
        model.add(Activation('sigmoid'))
        
        # Compile the model using the binary crossentropy loss and the Adam optimizer for it
        # We used the accuracy as a metric, but F1 score is also a plausible choice
        model.compile(loss='binary_crossentropy',
                      optimizer=Adam(lr=0.001),
                      metrics=['accuracy'])
            
        # Print a summary of the model to see what has been generated
        model.summary()
                      
        return model
    
    def train(self):
        
        # We define the number of epochs and steps per epochs
        EPOCHS = 150
        STEPS_PER_EPOCH = 1500
        
        # Early stopping callback after 10 steps
        early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=10, verbose=1, mode='auto')
        
        # Reduce learning rate on plateau after 4 steps
        lr_callback = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=4, verbose=1, mode='auto')
        
        # Place the callbacks in a list to be used when training
        callbacks = [early_stopping, lr_callback]
        
        # Train the model using the previously defined functions and callbacks
        self.model.fit_generator(create_minibatch(), steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS,\
                                 use_multiprocessing=False, workers=1, callbacks=callbacks, verbose=1)
    
    def classify(self, X):
        # Subdivide the images into blocks with a stride and patch_size of 16
        img_patches = create_patches(X, 16, 16, padding=28)
        
        # Predict
        predictions = self.model.predict(img_patches)
        predictions = (predictions[:,0] < predictions[:,1]) * 1
        
        # Regroup patches into images
        return group_patches(predictions, X.shape[0])
    
    def load(self, filename):
        # Load the model (used for submission)
        self.model = load_model(filename)
    
    def save(self, filename):
        # Save the model (used to then load to submit)
        self.model.save(filename)

In [18]:
# Instantiate the model with the size 72x72, the window size of the images to be fed
model = cnn_model(shape = (72, 72, 3))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 72, 72, 64)        4864      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 72, 72, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 36, 36, 128)       73856     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 36, 36, 128)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 128)       0         
__________

In [22]:
# Train the model
model.train()

Epoch 1/150
1500/1500 [==============================] - 258s 172ms/step - loss: 0.4145 - acc: 0.8093
Epoch 2/150
1500/1500 [==============================] - 237s 158ms/step - loss: 0.2569 - acc: 0.8961
Epoch 3/150
1500/1500 [==============================] - 238s 159ms/step - loss: 0.2346 - acc: 0.9083
Epoch 4/150
1500/1500 [==============================] - 238s 159ms/step - loss: 0.2218 - acc: 0.9153
Epoch 5/150
1500/1500 [==============================] - 238s 158ms/step - loss: 0.2157 - acc: 0.9181
Epoch 6/150
1500/1500 [==============================] - 237s 158ms/step - loss: 0.2125 - acc: 0.9204
Epoch 7/150
1500/1500 [==============================] - 236s 157ms/step - loss: 0.2099 - acc: 0.9215
Epoch 8/150
1500/1500 [==============================] - 237s 158ms/step - loss: 0.2084 - acc: 0.9224
Epoch 9/150
1500/1500 [==============================] - 236s 157ms/step - loss: 0.2058 - acc: 0.9240
Epoch 10/150
1500/1500 [==============================] - 237s 158ms/step - loss: 

## Saving the model

In [23]:
model.save('final_model.h5')